In [1]:
import pandas as pd

In [2]:
# upload file resume
resumes_origin = pd.read_csv(r"C:\Users\huuph\OneDrive\Documents\resume_matching\Resume_matching\Resume_Data.csv")
# upload file job
jobs_origin = pd.read_csv(r"C:\Users\huuph\OneDrive\Documents\resume_matching\Resume_matching\Job_Data.csv")
label_origin = pd.read_csv(r"C:\Users\huuph\OneDrive\Documents\resume_matching\Resume_matching\evaluation-labeled-data.csv")
# jobs_origin= jobs_origin[0:3]


In [3]:
list_resumes = []
for i in range(len(label_origin["name_job"])):
    list_resume = label_origin["resume"][i].split(", ")
    list_resumes.append([resumes_origin["Name"].iloc[int(j) -2] for j in list_resume])
label_origin["name_resume"] = list_resumes
label_origin

,name_job,resume,name_resume
0,Senior Software Developer.docx,"23, 9, 5, 19, 28, 15, 18, 21, 27, 48","[Nilesh Birari.docx, Debmalya Paul.docx, NAREN..."
1,Senior Product Manager.docx,"21, 24, 30, 5, 11, 23, 26, 31, 57, 58","[ABHAY RAJ DWIVEDI.docx, RISHABH SHARMA.docx, ..."
2,Data Scientist.docx,"8, 11, 4, 20, 42, 44, 47, 52, 54, 64","[DIVYA PRAKASH.docx, Amarpreet Singh.docx, ELI..."
3,HTML Developer.docx,"31, 5, 12, 15, 20, 21, 22, 23, 27, 43","[khyati gandhi.docx, NARENDRA SHUKLA.docx, Amr..."
4,IT Project Manager.docx,"23, 19, 21, 26, 28, 57, 58, 39, 55, 25","[Nilesh Birari.docx, Deepak Hariharan.docx, AB..."
5,Lead Technical Program Manager.docx,"55, 19, 21, 23, 28, 26, 43, 26, 30, 57","[Sneha Vijay Bhamkar.docx, Deepak Hariharan.do..."
6,Web_dev_job.docx,"31, 12, 27, 43, 22, 48, 61, 5, 28, 23","[khyati gandhi.docx, Amruta B.pdf, Dhavakumar...."
7,Backend Developer.docx,"15, 47, 38, 51, 11, 20, 23, 19, 28, 48","[Pooja L.docx, AJAY CHINNI.docx, Avik Bhattach..."
8,Web Developer.docx,"5, 7, 9, 12, 21, 22, 23, 27, 28, 31","[NARENDRA SHUKLA.docx, Ashwani Kumar Rajput.do..."
9,Revenue Reporting Data Analyst.docx,"4, 8, 28, 26, 19, 5, 7, 10, 20, 22","[ELIZABETH SONY THOMAS.docx, DIVYA PRAKASH.doc..."


In [4]:
def check_list(list1, list2):
    return [element for element in list1 if element in list2]

In [5]:
# extraction resume
from Job_resume_matching.extraction_resume import extraction_resume
info_retrieval_resume = extraction_resume(resumes_origin)
for i in range(len(info_retrieval_resume)):
        info_retrieval_resume["degrees"][i] = str([info_retrieval_resume["degrees"][i]])
        info_retrieval_resume["majors"][i] = str(info_retrieval_resume["majors"][i]) 
        info_retrieval_resume["skills"][i] = str(info_retrieval_resume["skills"][i])

In [6]:
from Job_resume_matching.extraction import extraction_jobs_all
info_retrieval_jobs = extraction_jobs_all(jobs_origin)


In [15]:
from Job_resume_matching.matching import matching, matching_evaluate
import yacs.config
import asyncio
from config import get_default_config
def load_config() -> yacs.config.CfgNode:
    config = get_default_config()
    return config

config = load_config()
_info_retrieval_jobs = info_retrieval_jobs.copy()
list_result_matching = []
_retreval_jobs = _info_retrieval_jobs.reset_index()
# for i in range(len(label_origin["name_job"])):
for i in range(0,2):
    _info_retrieval_resume = info_retrieval_resume.copy()
    print(i)
    retreval_jobs = _retreval_jobs.copy()
    _jobs = retreval_jobs[retreval_jobs["Name"] == label_origin["name_job"][i]]
    _jobs = _jobs.reset_index()
    results_matching = await asyncio.gather(matching_evaluate(_jobs,_info_retrieval_resume, config))
    list_result_matching.append(results_matching)
# results_matching


0
aa 1
1
aa 1


In [30]:
list_top_All_mpnet_base_v2 = []
list_top_Paraphrase_MiniLM_L6_v2 = []
list_top_All_MiniLM_L12_v1 = []   
for i in range(len(list_result_matching)):
    All_mpnet_base_v2 = list_result_matching[i][0][0].sort_values(by=["matching score job 0"], ascending=False).reset_index()
    top_name_resume_All_mpnet_base_v2 = All_mpnet_base_v2["Name"]
    list_top_All_mpnet_base_v2.append(top_name_resume_All_mpnet_base_v2[0:10])

    Paraphrase_MiniLM_L6_v2 = list_result_matching[i][0][1].sort_values(by=["matching score job 0"], ascending=False).reset_index()
    top_name_resume_Paraphrase_MiniLM_L6_v2 = Paraphrase_MiniLM_L6_v2["Name"]
    list_top_Paraphrase_MiniLM_L6_v2.append(top_name_resume_Paraphrase_MiniLM_L6_v2[0:10])
    
    All_MiniLM_L12_v1 = list_result_matching[i][0][2].sort_values(by=["matching score job 0"], ascending=False).reset_index()
    top_name_resume_All_MiniLM_L12_v1 = All_MiniLM_L12_v1["Name"]
    list_top_All_MiniLM_L12_v1.append(top_name_resume_All_mpnet_base_v2[0:10])


In [31]:
evaluate_All_mpnet_base_v2 = []
evaluate_Paraphrase_MiniLM_L6_v2 = []
evaluate_All_MiniLM_L12_v1 = []
for i in range(0,2):
    list_All_mpnet_base_v2 = list(list_top_All_mpnet_base_v2[i])
    list_Paraphrase_MiniLM_L6_v2 = list(list_top_Paraphrase_MiniLM_L6_v2[i])
    list_All_MiniLM_L12_v1 = list(list_top_All_MiniLM_L12_v1[i])
    list_2 = list(label_origin["name_resume"][i])
    check_score_1 = check_list(list_All_mpnet_base_v2, list_2)
    check_score_2 = check_list(list_Paraphrase_MiniLM_L6_v2, list_2)
    check_score_3 = check_list(list_All_MiniLM_L12_v1, list_2)
    evaluate_All_mpnet_base_v2.append(len(check_score_1)/10)
    evaluate_Paraphrase_MiniLM_L6_v2.append(len(check_score_2)/10)
    evaluate_All_MiniLM_L12_v1.append(len(check_score_3)/10)


In [32]:
data = {
    "All_mpnet_base_v2": evaluate_All_mpnet_base_v2,
    "Paraphrase_MiniLM_L6_v2": evaluate_Paraphrase_MiniLM_L6_v2,
    "All_MiniLM_L12_v1": evaluate_All_MiniLM_L12_v1
}
df = pd.DataFrame(data)
df

,All_mpnet_base_v2,Paraphrase_MiniLM_L6_v2,All_MiniLM_L12_v1
0,0.0,0.2,0.0
1,0.4,0.3,0.4


In [26]:
list_top_All_mpnet_base_v2[0]

0               Suraj Chauhan.docx
1            Prasanna Navnath.docx
2             Amarpreet Singh.docx
3                   Nandagopal.pdf
4           Avik Bhattacharya.docx
5              NARENDRA HINGE.docx
6               SARVESH KARAN.docx
7              GurshidPremium.docx
8               khyati gandhi.docx
9    Seshadri Sastry Kunapuli.docx
Name: Name, dtype: object

In [83]:
list1= list(results_matching[0][0]["Name"])
list2 = list(label_origin["name_resume"][0])
print(list1)
print(list2)
check_list(list1, list2)

['ELIZABETH SONY THOMAS.docx', 'NARENDRA SHUKLA.docx', 'Charu Tyagi.docx', 'Rahul Kumar.docx', 'Ishrat Fatma.docx']
['Nilesh Birari.docx', 'Debmalya Paul.docx', 'NARENDRA SHUKLA.docx', 'Deepak Hariharan.docx', 'Ranjan Raikwar.docx', 'Pooja L.docx', 'NEDA RIZVI.docx', 'ABHAY RAJ DWIVEDI.docx', 'Dhavakumar.docx', 'Renu Tiwari.docx']


['NARENDRA SHUKLA.docx']